### Objectiu: Predir si un pacient té una enfermetat cardiovascular en funció de les seves característiques clíniques.

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.preprocessing import LabelEncoder

dataset = pd.read_csv(".\\cardio_train.csv", sep=";")

print(dataset.describe())

X=dataset.drop('cardio',axis=1)
y=dataset['cardio']


                 id           age        gender        height        weight  \
count  70000.000000  70000.000000  70000.000000  70000.000000  70000.000000   
mean   49972.419900  19468.865814      1.349571    164.359229     74.205690   
std    28851.302323   2467.251667      0.476838      8.210126     14.395757   
min        0.000000  10798.000000      1.000000     55.000000     10.000000   
25%    25006.750000  17664.000000      1.000000    159.000000     65.000000   
50%    50001.500000  19703.000000      1.000000    165.000000     72.000000   
75%    74889.250000  21327.000000      2.000000    170.000000     82.000000   
max    99999.000000  23713.000000      2.000000    250.000000    200.000000   

              ap_hi         ap_lo   cholesterol          gluc         smoke  \
count  70000.000000  70000.000000  70000.000000  70000.000000  70000.000000   
mean     128.817286     96.630414      1.366871      1.226457      0.088129   
std      154.011419    188.472530      0.680250    

### Exploratory Data Analysis

Explorem la nostra base de dades, entenent quines són les característiques (dades que possiblement afectin al nostre objectiu) i l'objectiu (allò que volem predir). 

Variables de la base de dades:

0. id --> Índex de les mostres. Numèric quantitatiu continu.
1. age --> Edat del pacient (dies). Numèric quantitatiu discret.
2. height --> Alçada del pacient (cm). Numèric quantitatiu continu.
3. weight --> Pes del pacient (kg). Numèric quantitatiu continu.
4. gender --> Gènere del pacient (0, 1). Categòric qualitatiu binària
5. ap_hi --> Pressió arterial sistòlica del pacient (mmHg). Numèric quantitatiu discret.
6. ap_lo --> Pressió arterial diastòlica del pacient (mmHg). Numèric quantitatiu discret.
7. cholesterol --> Nivell de colesterol del pacient (1: normal, 2: per sobre del normal, 3: molt per sobre del normal). Categòric qualitatiu ordinal.
8. gluc --> Nivell de glucosa del pacient (1: normal, 2: per sobre del normal, 3: molt per sobre del normal). Categòric qualitatiu ordinal.
9. smoke --> Fumar o no (0: no, 1: sí). Categòric qualitatiu binari.
10. alco --> Consum d'alcohol (0: no, 1: sí). Categòric qualitatiu binari.
11. active --> Activitat física (0: no, 1: sí). Categòric qualitatiu binari.
12. cardio --> Presència o absència de malaltia cardiovascular (0: no, 1: sí). Categòric qualitatiu binari.

Tenim 12 característiques i 1 target ("cardio").

Volem predir "cardio" (presència o absència de malaltia cardiovascular en un pacient) en funció de totes les altres característiques.

In [18]:
print("\nMostrem els tipus de variables:\n")
print(X.dtypes)


Mostrem els tipus de variables:

id               int64
age              int64
gender           int64
height           int64
weight         float64
ap_hi            int64
ap_lo            int64
cholesterol      int64
gluc             int64
smoke            int64
alco             int64
active           int64
dtype: object


In [22]:
print("\nMostrem els primers valors de les característiques i de l'objectiu:\n")
print(X.head())


Mostrem els primers valors de les característiques i de l'objectiu:

   id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0   0  18393       2     168    62.0    110     80            1     1      0   
1   1  20228       1     156    85.0    140     90            3     1      0   
2   2  18857       1     165    64.0    130     70            3     1      0   
3   3  17623       2     169    82.0    150    100            1     1      0   
4   4  17474       1     156    56.0    100     60            1     1      0   

   alco  active  
0     0       1  
1     0       1  
2     0       0  
3     0       1  
4     0       0  


In [23]:
print(y.head())

0    0
1    1
2    1
3    1
4    0
Name: cardio, dtype: int64


Anem a veure si les nostres dades presenten NaNs:

In [29]:
print("\nNombre de NaNs per columna en els features:\n")
print(X.isna().sum())
print("\nPercentatge de NaNs per columna en els features:\n")
print(X.isna().mean() * 100)


Nombre de NaNs per columna en els features:

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
dtype: int64

Percentatge de NaNs per columna en els features:

id             0.0
age            0.0
gender         0.0
height         0.0
weight         0.0
ap_hi          0.0
ap_lo          0.0
cholesterol    0.0
gluc           0.0
smoke          0.0
alco           0.0
active         0.0
dtype: float64


In [30]:
print("\nNombre de NaNs per columna en el target:\n")
print(y.isna().sum())
print("\nPercentatge de NaNs per columna en el target:\n")
print(y.isna().mean() * 100)


Nombre de NaNs per columna en el target:

0

Percentatge de NaNs per columna en el target:

0.0


No tenim NaNs (valors buits) ni en el nostre objectiu ni en les característiques.